In [ ]:
# code below replaces entire DCI R workflow pre-2020
# uses two new output tables from FIPEX.
# FIPEX / ArcMap passes network data that includes nodes / junctions that are 
# branches in the river network (not just nodes that are barriers or sinks).
# DCIp, DCId, DCIs were all re-coded with alternatives benchmarked.
# Credit to Chris Edge (Natural Resources Canada) for DCId code that avoids loops in R.
# Please see accompanying Powerpoint for workflow diagrams. 
# - G Oldford, 2020

In [ ]:
# 'Advanced' table includes network edges and nodes
# with all original branch junctions in the GIS data 
# kept, even if they aren't barriers
# The 'advanced' table includes all info required to run DCI
# (previously two tables were required from FIPEX)
FIPEX_table=read.csv("FIPEX_Advanced_DD_2020.csv")
FIPEX_table

In [ ]:
# The params file allows users to pass param settings
# to R from within the ArcMap software (new in 2020)
FIPEX_params=read.csv("FIPEX_2020_params")
FIPEX_params

In [ ]:
# for testing: natural TF set node 84 to natural barrier
FIPEX_table[2,]$NaturalTF <- TRUE
FIPEX_table

In [ ]:
# required libraries
library(RBGL)
library(Rgraphviz)
library(rbenchmark)
library(data.table)
library(tidyverse)

# Jupyter R Notebooks can install packages required:
#install.packages("BiocManager")
#BiocManager::install("Rgraphviz")
#BiocManager::install("RBGL")

# new in 2020:
# (others may be used in experiemnts below but these are required as of 2020)
#install.packages("rbenchmark", repos='http://cran.us.r-project.org')
#install.packages("data.table", repos='http://cran.us.r-project.org')
#install.packages("tidyverse", repos='http://cran.us.r-project.org')

# Note 2020: 
# Some original files were modified to upgrade to newer version of R (3.6.1)
# variable names and functions with . were replaced with _ to avoid confusion 
# the visualization function 

## function: create_adjmatrix_2020

In [ ]:
###### CREATE ADJACENCY MATRIX #####
# rbgl requires adjacency matrix 
# FIPEX outputs adjacency table so need to convert
# note: igraph requires only adjacency table

# library(rbenchmark) can be used to test
#https://stackoverflow.com/questions/34355892/build-a-square-adjacency-matrix-from-data-frame-or-data-tablelevs <- unique(unlist(neighbournodes_all, use.names=F))

create_adjmatrix_2020 <- function(neighbournodes_all,option="2020_tapply") {
    # binary adjacency matrix defining connectivity
    
    if(option=="2020_tapply"){
        neighbournodes_all$edgeLength <- 1
        adj_matrix <- with(neighbournodes_all, tapply(edgeLength, list(Node1, Node2),FUN=length, default = 0))
        
    }else if(option=="2020_dfmatrix"){
        matrix_table <- table(neighbournodes_all)
        adj_mat1 <- as.data.frame.matrix(matrix_table)
        adj_matrix <- as.matrix(adj_mat1)
        
    }else if(option=="oldway"){
        
        # pre-2020 code
        segments<-with(neighbournodes_all, unique(Node2))
        #create a matrix with only 0's in them
        adj_matrix<-matrix(nrow=length(segments), 
                       ncol=length(segments), 
                       rep(0,length(segments)*length(segments)))
        segment_length<-length(segments)
        rownames(adj_matrix)<-colnames(adj_matrix)<-segments

        for (i in 1:segment_length){
            # find the segments in segment.matrix$Seg where segment.matrix$Seg_ID matches segments[i]
            # index of matching positions - this will be a vector of 1's and NA's
            pos_match<-	match(neighbournodes_all$Node2,segments[i])
        
            # keep only the positions where pos.match==1
            adj_segments<-neighbournodes_all$Node1[!is.na(pos_match)]

            # find the column positions that correspond to the adjacaent segments
            col<-match(adj_segments,segments)

            # the row number should correspond to i
            row<-i

            # assign a value of 1 for all values of row and col
            adj_matrix[row,col]<-1
        }
    }  
    adj_matrix
}

# check (need neighbournodes_all to check)
# dimensions should match
#dim(create_adjmatrix_2020(neighbournodes_all,"2020_tapply"))
#dim(create_adjmatrix_2020(neighbournodes_all,"2020_dfmatrix"))
#dim(create_adjmatrix_2020(neighbournodes_all,"oldway"))

In [ ]:
totalhabitatarea = sum(FIPEX_table$Quantity_Area)
totalhabitatlength = sum(FIPEX_table$Quantity_Len)
totallength = sum(FIPEX_table$DownstreamNeighDistance)
print("For cross checks in GIS:")
print(totalhabitatarea)
print(totalhabitatlength)
print(totallength)

### Experimental Section - connectivity list
#node oriented, for benchmarking only - next section has actual edge-oriented way accounting for length

In [ ]:
library(tidyverse)
library(data.table)

totalhabitatarea = sum(FIPEX_table$Quantity_Area)
totalhabitatlength = sum(FIPEX_table$Quantity_Len)
totallength = sum(FIPEX_table$DownstreamNeighDistance)

# ensure it's "sink" not "Sink"
FIPEX_table <- FIPEX_table %>%
mutate(DownstreamEID = ifelse(DownstreamEID == "Sink","sink",as.character(DownstreamEID)))


# get connectivity info only and convert columns to character,
FIPEX_table_reg <- FIPEX_table %>% select(NodeEID,DownstreamEID) %>%
mutate(temp = as.character(NodeEID)) %>%
mutate(NodeEID = temp) %>%
mutate(temp = as.character(DownstreamEID)) %>%
mutate(DownstreamEID = temp) %>%
select(-temp) 

# produce connectivity table (node and neighbour node pairs)
# downstream neighbours
neighbournodes_down <- FIPEX_table_reg

# self-connected
neighbournodes_self <- FIPEX_table_reg %>% select(NodeEID,DownstreamEID) %>%
mutate(DownstreamEID = NodeEID)

# upstream neigbours
neighbournodes_up <- FIPEX_table_reg %>% select(NodeEID,DownstreamEID) %>%
mutate(temp = DownstreamEID) %>%
mutate(DownstreamEID = NodeEID) %>%
mutate(NodeEID = temp) %>%
select(NodeEID, DownstreamEID)

# merge
neighbournodes_all <- neighbournodes_down %>% bind_rows(neighbournodes_self) %>%
bind_rows(neighbournodes_up) %>%
add_row(NodeEID = "sink", DownstreamEID = "sink") %>%
rename(Node1 = NodeEID, Node2 = DownstreamEID)
neighbournodes_all

In [ ]:
# benchmark ways of creating adjacency matrix
# using 'regular' (non-dd) way
library(rbenchmark)
benchmark(create_adjmatrix_2020(neighbournodes_all,"2020_tapply"),
create_adjmatrix_2020(neighbournodes_all,"2020_dfmatrix"),
create_adjmatrix_2020(neighbournodes_all,"oldway"),
replications=1000)
# tapply fastest

In [ ]:
################################################################
# produce binary connectivity matrix - useful for regular DCI
# zeros on diagonals are for visualization

adj_mat_new <- create_adjmatrix_2020(neighbournodes_all,"2020_tapply")
#adj_mat_new <- create_adjmatrix_2020(neighbournodes_all,"2020_dfmatrix"),
#adj_mat_new <- create_adjmatrix_2020(neighbournodes_all,"oldway"

# produce copy with zeros on diagonal for visuals
adj_matrix_zeros_on_diag<- adj_mat_new
# diag() is a core R function - GO
diag(adj_matrix_zeros_on_diag)<-0

### Create Edge-Weighted Connectivity Table 

In [ ]:
################################################################
# edge-weighted connectivity list and matrix - for DCI w/ DD
# nodes / edges are not reversed as in original way
# do not need self-connected

# get connectivity list, convert columns to characters
FIPEX_table_DD <- FIPEX_table %>% select(NodeEID,DownstreamEID,DownstreamNeighDistance) %>%
mutate(temp = as.character(NodeEID)) %>%
mutate(NodeEID = temp) %>%
mutate(temp = as.character(DownstreamEID)) %>%
mutate(DownstreamEID = temp) %>%
select(-temp) %>%
mutate(DownstreamEID = ifelse(DownstreamEID == "Sink","sink",DownstreamEID))

# downstream neighbours
edges_down <- FIPEX_table_DD

# upstream neigbours
edges_up <- FIPEX_table_DD %>% select(NodeEID,DownstreamEID,DownstreamNeighDistance) %>%
mutate(temp = DownstreamEID) %>%
mutate(DownstreamEID = NodeEID) %>%
mutate(NodeEID = temp) %>%
select(-temp)

# self-connected
#FIPEX_table %>% select(NodeEID,DownstreamEID,DownstreamNeighDistance) %>%
#mutate(NodeEID = DownstreamEID, DownstreamNeighDistance = 0.1)

# merge
edges_all <- edges_down %>%
bind_rows(edges_up) %>%
#add_row(NodeEID = "sink", DownstreamEID = "sink") %>%
rename(Node1 = NodeEID, Node2 = DownstreamEID, edgeLength = DownstreamNeighDistance)

adj_matrix_edgelengths <- with(edges_all, tapply(edgeLength, list(Node1, Node2), FUN=sum, default = 0))

In [ ]:
edges_all

In [ ]:
adj_matrix_edgelengths

## Function: create_graph_dd_2020

In [ ]:
#to do: alternative graph creation using igraph
# benchmark
# note most code below assumes using an RGBL Boost GraphAM object,
#  so to test alternative it would require a lot of alternative code,
#  potentially

In [ ]:
################################################################
# CREATE GRAPH OBJECT for DCI w/ Distance Decay
# edge weights are used for distance calculations
# edge data / attributes used for habitat quantity calculations
# #https://www.rdocumentation.org/packages/graph/versions/1.50.0/topics/graphAM-class
create_graph_dd_2020 <- function(adj_matrix_edgelengths=0.0,FIPEX_table=NULL){

    # Create graph object
    # 2020 - different way to call the graphAM function 
    # vs pre-2020
    g_dd <- graphAM(adjMat=adj_matrix_edgelengths,  edgemode="directed", values=list(weight=1))

    # associate passabilities with nodes using NodeData slot
    # e.g. nodeData(g,n=c("b", "c"), attr ="color") <- "red"
    nodeDataDefaults(g_dd, attr ="pass") <- 1.0
    nodeData(g_dd,n=as.character(FIPEX_table$NodeEID), attr="pass") <- as.double(FIPEX_table$BarrierPerm)
    #nd <- nodes(g_dd)

    nodeDataDefaults(g_dd, attr ="nodelabel") <- "none"
    nodeData(g_dd,n=as.character(FIPEX_table$NodeEID), attr="nodelabel") <- as.character(FIPEX_table$NodeLabel)
    nodeData(g_dd,n="sink", attr="nodelabel") <- "sink"
    #nd <- nodes(g_dd)

    nodeDataDefaults(g_dd, attr ="downnodelabel") <- "none"
    nodeData(g_dd,n=as.character(FIPEX_table$NodeEID), attr="downnodelabel") <- as.character(FIPEX_table$DownstreamNodeLabel)
    #nd <- nodes(g_dd)

    nodeDataDefaults(g_dd, attr ="natural") <- "none"
    nodeData(g_dd,n=as.character(FIPEX_table$NodeEID), attr="natural") <- FIPEX_table$NaturalTF
    nodeData(g_dd,n="sink", attr="natural") <- FALSE
    #nd <- nodes(g_dd)

    # optionally can give edges attributes
    #edgeDataDefaults(g_dd, attr="name")<-"noname"
    #edgeData(self, from, to, attr)
    #edgeData(self, from, to, attr) <- value
    edgeDataDefaults(g_dd, attr="HabLen")<-0.0
    edgeData(g_dd,from=as.character(FIPEX_table$NodeEID), 
         to=as.character(FIPEX_table$DownstreamEID), 
         attr="HabLen")<-as.double(FIPEX_table$Quantity_Len)
    # reverse - attr associated with each direction along one edge
    edgeData(g_dd,from=as.character(FIPEX_table$DownstreamEID), 
         to=as.character(FIPEX_table$NodeEID), 
         attr="HabLen")<-as.double(FIPEX_table$Quantity_Len)

    edgeDataDefaults(g_dd, attr="HabArea")<-0.0
    edgeData(g_dd,from=as.character(FIPEX_table$NodeEID), 
         to=as.character(FIPEX_table$DownstreamEID), 
         attr="HabArea")<-as.double(FIPEX_table$Quantity_Area)
    # reverse - attr associated with each direction along one edge
    edgeData(g_dd,from=as.character(FIPEX_table$DownstreamEID), 
         to=as.character(FIPEX_table$NodeEID), 
         attr="HabArea")<-as.double(FIPEX_table$Quantity_Area)

    # give edges an easy-to-access name insensitive to direction
    # this is done to quickly identify duplicates later
    # there may be alternatives such as accessing edgeNames but I suspect
    # they are slower than this
    edgeDataDefaults(g_dd, attr="EdgeNameGO")<-"init"
    edgeData(g_dd,from=as.character(FIPEX_table$NodeEID), 
         to=as.character(FIPEX_table$DownstreamEID), 
         attr="EdgeNameGO")<-paste(as.character(FIPEX_table$DownstreamEID),
                                   as.character(FIPEX_table$NodeEID),
                                   sep="-")
    # reverse - attr associated with each direction along one edge
    edgeData(g_dd,from=as.character(FIPEX_table$DownstreamEID), 
         to=as.character(FIPEX_table$NodeEID), 
         attr="EdgeNameGO")<-paste(as.character(FIPEX_table$DownstreamEID),
                                   as.character(FIPEX_table$NodeEID),
                                   sep="-")
    return(g_dd)
}
g_dd<-create_graph_dd_2020(adj_matrix_edgelengths,FIPEX_table)

In [ ]:
# note ignore the bidirectional arrows
# to do: add visual function
library(Rgraphviz)

plot(g_dd)

## Function: get_paths_distances

In [ ]:
################################################################################
# get all distances and paths (from penult) to all nodes from Sink / all nodes
# https://www.rdocumentation.org/packages/RBGL/versions/1.48.1/topics/dijkstra.sp
# note it must be node-node not edge-edge
# note using this function repeatedly during DCIp is inefficient 
# !!! (should use BFS w/ LCA i.e., custom algorithm) !!!

get_paths_distances <- function(g=NULL,fromnode="sink"){
    dijkstra.sp(g,fromnode,eW=unlist(edgeWeights(g)))
    
    # TO DO: ALTERNATIVES FOR BENCHMARKING
}

In [ ]:
# Example useage
paths_distances_sink <- get_paths_distances(g_dd,"sink")
paths_distances_sink$distances["100"]

## Function: get_summary_tab_2020

In [ ]:
##############################################################################
##### SUMMARY TABLE 2020 #####

# replaces similar pre-2020 function to create a table for each edge-edge pair.
# code includes options for alternative data management for benchmarking. 
# this function could be sped up with custom algorithm that can find path 
# while also grabbing attribute data. 

# gets cumulative passability each pair using path info
# and get other attributes
#   (algo is not most efficient but cannot edit the 
#     source for Djikstra.sp because it's actually an interface to C++ 'Boost'
#     library for graphs - can't get edge and node attributes during net traversal)

# pseudocode:
# for each 'from node' (e.g., sink in DCId, and all nodes in DCIp)
#  get paths between node and all other node
#
#  for each 'to node' in 'all paths' results
#   get the first edge len and hab traversed from node to sink
#
#   store length and hab of the edge between 'to node' and first node encountered
#   in path back to 'from node' (i.e., the 'to edge')

#   do while next node name <> "sink"
#     pass = nodeData(g_dd, nextnode, "pass")
#     cumulativepass =  cumulativepass * pass
#     nextnode = the next node in path towards sink
#     if last edge traversed on the way to sink
#       store the length and habitat of this edge which is the 'from edge'
#   
#   add various other attributes to master table (attr's stored in g object)

# requires library(data.table)
# data.table vs other options likely to speed things up for large networks
#https://rstudio-pubs-static.s3.amazonaws.com/406521_7fc7b6c1dc374e9b8860e15a699d8bb0.html
#https://www.rdocumentation.org/packages/data.table/versions/1.13.0/topics/rbindlist

get_summary_tab_2020 <- function(option="dt-lists",
                                naturalonly=FALSE,
                                g = NULL,
                                DCIp=FALSE){
    # funciton params:
    # option - for benchmarking speed of appending to table
    # naturalonly - will calculate pass-weighted path distances
    #  while ignoring non-natural barriers
    # g - the graph object (rbgl GraphAM in BioconductR)
    # DCIp - TRUE / FALSE will trigger loop that finds path
    #       between all nodes, node just sink
    # initialize empty data object in different ways
    
    # for different options and benchmarking:
    DT2 = data.table(FromNode="init",
                 ToNode="init",
                 FromNodeLabel="init",
                 ToNodeLabel="init",
                 CumulativePass=0.0,
                 FromEdgeLen=0.0,
                 ToEdgeLen=0.0,
                 TotalDist=0.0,
                 DistMinusStartEndLen=0.0,
                 FromEdgeHabLen=0.0,
                 ToEdgeHabLen=0.0,
                 FromEdgeHabArea=0.0,
                 ToEdgeHabArea=0.0,
                 ToEdgeName="init",
                 FromEdgeName="init",
                 ToFromEdgeNameCombo="init")
    
    DF2 = data.frame(FromNode="init",
                 ToNode="init",
                 FromNodeLabel="init",
                 ToNodeLabel="init",
                 CumulativePass=0.0,
                 FromEdgeLen=0.0,
                 ToEdgeLen=0.0,
                 TotalDist=0.0,
                 DistMinusStartEndLen=0.0,
                 FromEdgeHabLen=0.0,
                 ToEdgeHabLen=0.0,
                 FromEdgeHabArea=0.0,
                 ToEdgeHabArea=0.0,
                 ToEdgeName="init",
                 FromEdgeName="init",
                 ToFromEdgeNameCombo="init", stringsAsFactors=F)
    
    # lists in R must have size pre-allocated 
    # size of our table is almost n^2 - n*(n-1) 
    # (less than n^2 since not getting distance from node to itself)
    if(DCIp==FALSE){
        outlist <- vector("list", length(numNodes(g_dd)))
    }else{
        outlist <- vector("list", length(numNodes(g_dd)*(numNodes(g_dd)-1)))   
    }
    outlist[[numNodes(g_dd)]] <- list(FromNode="init",
                 ToNode="init",
                 FromNodeLabel="init",
                 ToNodeLabel="init",
                 CumulativePass=0.0,
                 FromEdgeLen=0.0,
                 ToEdgeLen=0.0,
                 TotalDist=0.0,
                 DistMinusStartEndLen=0.0,
                 FromEdgeHabLen=0.0,
                 ToEdgeHabLen=0.0,
                 FromEdgeHabArea=0.0,
                 ToEdgeHabArea=0.0,
                 FromEdgeName="init",
                 ToEdgeName="init",
                 ToFromEdgeNameCombo="init")

    # from = sink / start node
    # to = other nodes
    if(DCIp==FALSE){
        fromnodecount=1
    }else{
        fromnodecount=numNodes(g_dd)
    }
    
    count = 0
    for (j in 1:fromnodecount){
    #indendation for two nested for loops not done to save space
        if(DCIp==FALSE){
            fromnode_name = "sink"
            fromnode_label = "sink" 
        }else{
            fromnode_name = nodes(g_dd)[j]
             if (fromnode_name=="sink"){
                fromnode_label = "sink"
            }else{
                fromnode_label = nodeData(g_dd, fromnode_name, "nodelabel")[[1]]  
            }
        }
        
        ###########################################################
        #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        # get path & distances between 'fromnode' and all other nodes
        paths_distances <- get_paths_distances(g,fromnode_name)
        # this can be time consuming
        
    for (k in 1:length(paths_distances$penult)) {
        
        tonode <- paths_distances$penult[k]     
        tonode_name = names(tonode)
        tonode_name <- tonode_name[[1]]
        tonode_label = nodeData(g_dd, tonode_name, "nodelabel")[[1]]
    
        if (tonode_name == fromnode_name){
            # not interested in distance from one node to itself
            next
        }
        count = count+1
        # initialize
        cumulativepass = 1.0
        pass = 1.0 # watch not to take pass from to/from end nodes since traversal starts at edge
        totaldistance = paths_distances$distances[tonode_name]
        totaldistance <- totaldistance[[1]]     
        
        # get length of edge 
        nextnode = paths_distances$penult[tonode]
        nextnode_name = names(nextnode)
        lastnode_name = tonode_name
    
        # get the last edge length traversed on the way to 'to node'
        # alternatively could grab the weight for this edge instead of subtraction
        toedgelen = totaldistance - paths_distances$distances[nextnode_name]
        toedgelen <- toedgelen[[1]]
        toedgedata =edgeData(g_dd, tonode_name,nextnode_name)
        toedgehablen = toedgedata[[1]]$HabLen
        toedgehabarea = toedgedata[[1]]$HabArea
        toedgename = toedgedata[[1]]$EdgeNameGO
        
        exitvar = "go"
        while (exitvar != "stop"){
        
            if(nextnode_name != fromnode_name){
                pass = nodeData(g_dd, nextnode_name, "pass")
                if(naturalonly==FALSE){
                    cumulativepass = cumulativepass * pass[[1]]
                }else{
                    natural = nodeData(g_dd,nextnode_name,"natural")
                    if(natural[[1]]==TRUE){
                        cumulativepass = cumulativepass * pass[[1]]
                    }
                }
            }else{
                fromedgelen = paths_distances$distances[lastnode_name]
                fromedgelen <- fromedgelen[[1]]
                fromedgedata = edgeData(g_dd, lastnode_name,fromnode_name)
                fromedgehablen = fromedgedata[[1]]$HabLen
                fromedgehabarea = fromedgedata[[1]]$HabArea
                fromedgename = fromedgedata[[1]]$EdgeNameGO
                
                exitvar="stop"
            }
            
            lastnode_name = nextnode_name
            nextnode = paths_distances$penult[nextnode]
            nextnode_name = names(nextnode)
        }
        
        distminusstartendlen = totaldistance - toedgelen - fromedgelen
       tofromedgename_combo = paste(toedgename,fromedgename,sep="|")
        
        if (option=="dt"){
            #print(cumulativepass)
            #https://www.rdocumentation.org/packages/data.table/versions/1.13.0/topics/rbindlist
            DT1 = data.table(FromNode=fromnode_name,
                     ToNode=tonode_name,
                     FromNodeLabel=fromnode_label,
                     ToNodeLabel=tonode_label,
                     CumulativePass=cumulativepass, 
                     FromEdgeLen=fromedgelen,
                     ToEdgeLen=toedgelen,
                     TotalDist=totaldistance,
                     DistMinusStartEndLen=distminusstartendlen,
                     FromEdgeHabLen=fromedgehablen,
                     ToEdgeHabLen=toedgehablen,
                     FromEdgeHabArea=fromedgehabarea,
                     ToEdgeHabArea=toedgehabarea,
                     FromEdgeName=fromedgename,
                     ToEdgeName=toedgename,
                     ToFromEdgeNameCombo=tofromedgename_combo)
            l = list(DT1,DT2)
            
            DT2 = rbindlist(l, use.names=TRUE)
        }else if(option=="dt-lists"){
            # append lists to list rather than work yet with tables
            DL1 = list(FromNode=fromnode_name,
                     ToNode=tonode_name,
                     FromNodeLabel=fromnode_label,
                     ToNodeLabel=tonode_label,
                     CumulativePass=cumulativepass, 
                     FromEdgeLen=fromedgelen,
                     ToEdgeLen=toedgelen,
                     TotalDist=totaldistance,
                     DistMinusStartEndLen=distminusstartendlen,
                     FromEdgeHabLen=fromedgehablen,
                     ToEdgeHabLen=toedgehablen,
                     FromEdgeHabArea=fromedgehabarea,
                     ToEdgeHabArea=toedgehabarea,
                     FromEdgeName=fromedgename,
                     ToEdgeName=toedgename,
                     ToFromEdgeNameCombo=tofromedgename_combo)
            #print("Length DL1: ")
            #print(length(DL1))
            outlist[[count]] <- (DL1)
            
        }else if(option=="df"){
            DF1 = data.frame(FromNode=fromnode_name,
                     ToNode=tonode_name,
                     FromNodeLabel=fromnode_label,
                     ToNodeLabel=tonode_label,
                     CumulativePass=cumulativepass, 
                     FromEdgeLen=fromedgelen,
                     ToEdgeLen=toedgelen,
                     TotalDist=totaldistance,
                     DistMinusStartEndLen=distminusstartendlen,
                     FromEdgeHabLen=fromedgehablen,
                     ToEdgeHabLen=toedgehablen,
                     FromEdgeHabArea=fromedgehabarea[[1]],
                     ToEdgeHabArea=toedgehabarea,
                     FromEdgeName=fromedgename,
                     ToEdgeName=toedgename,
                     ToFromEdgeNameCombo=tofromedgename_combo)
            #print(fromedgehabarea)
            #print(DF1)
            DF2 <- rbind(DF2, DF1)
          }else if(option=="df-lists"){
            DL1 = list(FromNode=as.character(fromnode_name),
                     ToNode=as.character(tonode_name),
                     FromNodeLabel=as.character(fromnode_label),
                     ToNodeLabel=as.character(tonode_label),
                     CumulativePass=as.numeric(cumulativepass), 
                     FromEdgeLen=as.numeric(fromedgelen),
                     ToEdgeLen=as.numeric(toedgelen),
                     TotalDist=as.numeric(totaldistance),
                     DistMinusStartEndLen=as.numeric(distminusstartendlen),
                     FromEdgeHabLen=as.numeric(fromedgehablen),
                     ToEdgeHabLen=as.numeric(toedgehablen),
                     FromEdgeHabArea=as.numeric(fromedgehabarea),
                     ToEdgeHabArea=as.numeric(toedgehabarea),
                     FromEdgeName=as.character(fromedgename),
                     ToEdgeName=as.character(toedgename),
                     ToFromEdgeNameCombo=as.character(tofromedgename_combo))
            outlist[[count]] <- (DL1)
        }else if(option=="dplyr"){
             DL1 = list(FromNode=as.character(fromnode_name),
                     ToNode=as.character(tonode_name),
                     FromNodeLabel=as.character(fromnode_label),
                     ToNodeLabel=as.character(tonode_label),
                     CumulativePass=as.numeric(cumulativepass), 
                     FromEdgeLen=as.numeric(fromedgelen),
                     ToEdgeLen=as.numeric(toedgelen),
                     TotalDist=as.numeric(totaldistance),
                     DistMinusStartEndLen=as.numeric(distminusstartendlen),
                     FromEdgeHabLen=as.numeric(fromedgehablen),
                     ToEdgeHabLen=as.numeric(toedgehablen),
                     FromEdgeHabArea=as.numeric(fromedgehabarea),
                     ToEdgeHabArea=as.numeric(toedgehabarea),
                     FromEdgeName=fromedgename,
                     ToEdgeName=toedgename,
                     ToFromEdgeNameCombo=tofromedgename_combo)
            #print(row1)
            DF2 <- bind_rows(DF2,DL1)
        }
    } #k
    } #j
        
    if(option=="dt"){
        
        DT2
    }else if(option=="dt-lists"){
        DT2 <- data.table(rbindlist(outlist))
        
        DT2
    }else if(option=="df"){
        #DF2 <- DF2[!duplicated(DF2$ToFromEdgeNameCombo), ]
        DF2
    }else if(option=="df-lists"){
        DF2 <- data.frame(do.call(rbind, outlist))
        #DF2 <- DF2[!duplicated(DF2$ToFromEdgeNameCombo), ]
        DF2
    }else if(option=="dplyr"){
        DF2
    }
} #function


In [ ]:
# example useage
# warning 'df-lists' does not work - it creates list type columns
#    which will cause errors in DCI calc!
sum_tab_2020 <- get_summary_tab_2020(option="dt-lists",
                     naturalonly=FALSE,
                     g = g_dd,
                     DCIp=TRUE
                    )
sum_tab_2020 

In [ ]:
class(sum_tab_2020) # note data tables are both df and dt

In [ ]:
### benchmarking different approaches to summary table creation
# test with a large (1000's nodes) dataset 
library(rbenchmark)

benchmark(get_summary_tab_2020(option="dt",
                     naturalonly=FALSE,
                     g = g_dd,
                     DCIp=TRUE),
          get_summary_tab_2020(option="dt-lists",
                     naturalonly=FALSE,
                     g = g_dd,
                     DCIp=TRUE),
          get_summary_tab_2020(option="df",
                     naturalonly=FALSE,
                     g = g_dd,
                     DCIp=TRUE),
          get_summary_tab_2020(option="df-lists",
                     naturalonly=FALSE,
                     g = g_dd,
                     DCIp=TRUE),
          get_summary_tab_2020(option="dplyr",
                     naturalonly=FALSE,
                     g = g_dd,
                     DCIp=TRUE),
          replications=40)
#dt-lists fastest

## Functions for DCI calculation

In [ ]:
#options: DCIp, DCIs, dd (yes/no), cutoff_dist, dd_equation

########################################################
###### Calculate DCI #####
#dci_calc_2020_dd <- function(){}
# warning: the sum_tab_2020 must be a data.table
#          only some data frames work ('dplyr' is ok not
#          the 'df-lists' option)

naturalonly = FALSE
DCIp = FALSE
DCIs = FALSE

#### (1) DCId - calc_DCId ####
calc_DCId_2020 <- function(sum_tab_2020=NULL,
                      FromNode="sink"){
    
    # filter table
    DCId_data<-subset(sum_tab_2020,FromNode=="sink")
    
    # Credit to Chris Edge Code for avoiding loops in R here: 
    DCId_data$temp <- DCId_data$CumulativePass * (DCId_data$ToEdgeHabLen/totalhabitatlength)
    DCId <- sum(DCId_data$temp)
    DCId
} # DCId

#### (2) DCIp - calc_DCIp ####
# Credit to C Edge for shorter code
calc_DCIp_2020 <- function(sum_tab_2020=NULL,
                      option="unique"){
    
    # 'option' = unique / distinct for benchmarking speeds
    # unique = data.table / base r
    # distinct = dplyr
    # sometimes sum_tab may be a data.table sometimes data.frame
    
    if(option=="unique"){
        sum_tab_2020 <- unique(sum_tab_2020, by = "ToFromEdgeNameCombo")
    }else{
        sum_tab_2020 <- distinct(sum_tab_2020, ToFromEdgeNameCombo, .keep_all = TRUE)
    }

    DCIp <- 0
    for (i in 1:nrow(sum_tab_2020)) {
        DCIp <- DCIp + (sum_tab_2020$CumulativePass[i] * (sum_tab_2020$ToEdgeHabLen[i]/totalhabitatlength) * (sum_tab_2020$FromEdgeHabLen[i]/totalhabitatlength))
    }
    DCIp
}

#### (3) DCIs - calc_DCIs ####
# can be added into (2) as option see below
calc_DCIs_2020 <- function (sum_tab_2020=NULL,totalhabitatlength=0.0,option="dt"){
    
    # option: "dt","dplyr","old"
    # alternative methods for benchmarking
    if(option=="dt"){
        
        DCIs <- sum_tab_2020
        # remove duplicates
        DCIs <- unique(sum_tab_2020, by = "ToFromEdgeNameCombo")
        # select only a required columns
        cols = c("FromEdgeName","ToEdgeHabLen","CumulativePass")
        DCIs <- DCIs[,..cols]
        # first step to DCIs
        DCIs[, DCIs_i := (ToEdgeHabLen/totalhabitatlength * CumulativePass)]
        cols = c("FromEdgeName","DCIs_i")
        DCIs <- DCIs[,..cols]
        # second step to DCIs
        DCIs <- DCIs[, lapply(.SD,sum), by=.(FromEdgeName)]
        
    }else if(option=="dplyr"){
        
        DCIs <- sum_tab_2020 %>%
        distinct(ToFromEdgeNameCombo, .keep_all = TRUE) %>%
        mutate(DCIs_i = CumulativePass * ToEdgeHabLen/totalhabitatlength) %>%
        select(DCIs_i,FromEdgeName) %>%
        group_by(FromEdgeName) %>%
        summarise(DCIs = sum(DCIs_i))

        
    }else if(option=="old"){
        sum_tab_2020 <- unique(sum_tab_2020, by = "ToFromEdgeNameCombo")
        sections<-as.vector(unique(sum_tab_2020$FromEdgeName))
        # store the all section results in DCI.as
        DCI_as<-NULL
        
        for(s in 1:length(sections)){
            DCI_s<-0
            # Old notes:
            # select out only the data that corresponds to pathways from one sectino 
            # to all other sections
            d_nrows<-subset(sum_tab_2020, FromEdgeName==sections[s])
            d_sum_table<-d_nrows
            
            for (a in 1:dim(d_nrows)[1]){
                # Old note:
                #to get the DCI for diadromous fish, use the following formula: 
                # DCId= li/L*Cj (where j= the product of the passability in the pathway)
                la<-d_sum_table$ToEdgeHabLen[a]/sum(FIPEX_table$Quantity_Len)
                pass_d<-d_sum_table$CumulativePass[a]
                DCI_s<-round(DCI_s+la*pass_d*100, digits=2)
            } # end loop over sections for dci calc
        
            DCI_as[s]<-DCI_s
        } # end loop over "first" sections	

        # STORE RESULTS IN .CSV file
        DCIs<-data.frame(sections,DCI_as)
    }else{
        print("error in options passed to calc_DCIs")
        DCIs <- 0.0
    }
    return(DCIs)
  
}


In [ ]:
#example useage
calc_DCId_2020(sum_tab_2020,"sink")
calc_DCIp_2020(sum_tab_2020,"unique")
calc_DCIs_2020(sum_tab_2020,totalhabitatlength,"dt")

### Crosschecks and benchmarking (redundant code to above)

In [ ]:
# crosscheck DCIp (this is old way of calculating)
# summary tab csv must be based on same network config!
OLD_table=read.csv("summary_table_all.csv")

DCIp <- 0
for (i in 1:nrow(OLD_table)) {
    DCIp <- DCIp + (OLD_table$pathway_pass[i] * (OLD_table$start_section_length[i]/totalhabitatlength) * (OLD_table$finish_section_length[i]/totalhabitatlength))
}
print(DCIp)
#totalhabitatlength
#OLD_table$start_section_lengt

In [ ]:
# 
library(rbenchmark)

benchmark(calc_DCIp(sum_tab_2020,"unique"),
          calc_DCIp(sum_tab_2020,"distinct"),
          replications=1000)
# unique is faster (data.table) vs distinct (dplyr)

In [ ]:
# benchmarking and crosschecking - I tested three or four ways of calculating DCI_s. 
# pre-2020 was very slow, data.table fastest as expected

calc_DCIs_dplyr <- function(sum_tab_2020=NULL,totalhabitatlength=NULL){
    
    DCIs <- sum_tab_2020 %>%
    distinct(ToFromEdgeNameCombo, .keep_all = TRUE) %>%
    mutate(DCIs_i = CumulativePass * ToEdgeHabLen/totalhabitatlength) %>%
    select(DCIs_i,FromEdgeName) %>%
    group_by(FromEdgeName) %>%
    summarise(DCIs = sum(DCIs_i))

    DCIs
}

calc_DCIs_dt <- function(sum_tab_2020=NULL,totalhabitatlength=NULL){
    
    DCIs <- sum_tab_2020
    # remove duplicates
    DCIs <- unique(sum_tab_2020, by = "ToFromEdgeNameCombo")
    # select only a required columns
    cols = c("FromEdgeName","ToEdgeHabLen","CumulativePass")
    DCIs <- DCIs[,..cols]
    # first step to DCIs
    DCIs[, DCIs_i := (ToEdgeHabLen/totalhabitatlength * CumulativePass)]
    cols = c("FromEdgeName","DCIs_i")
    DCIs <- DCIs[,..cols]
    # second step to DCIs
    DCIs[, lapply(.SD,sum), by=.(FromEdgeName)]

}

calc_DCIs_old <- function(sum_tab_2020=NULL,totalhabitatlength=NULL){

    sum_tab_2020 <- unique(sum_tab_2020, by = "ToFromEdgeNameCombo")
    sections<-as.vector(unique(sum_tab_2020$FromEdgeName))
    # store the all section results in DCI.as
    DCI_as<-NULL
        
    for(s in 1:length(sections)){
        DCI_s<-0
        # Old notes:
        # select out only the data that corresponds to pathways from one sectino 
        # to all other sections
        d_nrows<-subset(sum_tab_2020, FromEdgeName==sections[s])
        d_sum_table<-d_nrows
            
        for (a in 1:dim(d_nrows)[1]){
            # Old note:
            #to get the DCI for diadromous fish, use the following formula: 
            # DCId= li/L*Cj (where j= the product of the passability in the pathway)
            la<-d_sum_table$ToEdgeHabLen[a]/sum(FIPEX_table$Quantity_Len)
            pass_d<-d_sum_table$CumulativePass[a]
            DCI_s<-round(DCI_s+la*pass_d*100, digits=2)
        } # end loop over sections for dci calc
        
        DCI_as[s]<-DCI_s
    } # end loop over "first" sections	

    # STORE RESULTS IN .CSV file
    res<-data.frame(sections,DCI_as)
    res
}


# cross check new way / old way of DCIs
calc_DCIs_dplyr(sum_tab_2020,totalhabitatlength)
calc_DCIs_dt(sum_tab_2020,totalhabitatlength)
calc_DCIs_old(sum_tab_2020,totalhabitatlength)

In [ ]:
# 
library(rbenchmark)

benchmark(calc_DCIs_dplyr(sum_tab_2020,totalhabitatlength),
          calc_DCIs_dt(sum_tab_2020,totalhabitatlength),
          calc_DCIs_old(sum_tab_2020,totalhabitatlength)
          replications=30)
# unique is faster (data.table) vs distinct (dplyr)

In [ ]:
# correct output for DCIs from FIPEX for test network
83_s	28.78
84_s	4.5
96_s	28.78
98_s	4.69
94_s	5.76
97_s	10.65
99_s	29.93
95_s	5.76
100_s	15.69
102_s	29.93
101_s	15.69
sink	28.78